In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

# Positional Encodings

Wie Ihr gerade gesehen habt erzeugen wir aus jedem Wort einen Vektor mit 512 Dimensionen.

Da Wörter, bzw. deren vektorielle Repräsentation unterschiedliche Wahrnehmungen erzeugen, 
je nachdem welche Position diese Wörter im Satz einnehmen, möchten wir diese Position erfassen. 

Daher addieren wir zu den Embeddings (vektorielle Darstellung einer Sequenz) jeweils Vektoren mit der Länge 512. 


Diese Positionsvektoren sind immer gleich und folgen wie bereits erwähnt folgendem Schema: 

Für gerade Positionen:
$$
PE(pos, 2i) = \sin \left( \frac{pos}{10000^{\frac{2i}{d_{model}}}} \right)
$$

Für ungerade Positionen:
$$
PE(pos, 2i + 1) = \cos \left( \frac{pos}{10000^{\frac{2i}{d_{model}}}} \right)
$$

### Aufgabe 1

Vervollständige __#1__ und __#2__. 

__Hinweis: Die Sinusfunktion in pytorch kannst du mit torch.sin() und die Cosinusfunktion mit torch.cos() aufrufen.__



In [2]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        # Erzeugung einer Nullmatrix pe mit den Dimensionen max_seq_length x d_model
        pe = torch.zeros(max_seq_length, d_model)

        # Erzeugung eines Tensor 'position' mit Werten von 0 bis max_seq_length-1 und Umwandlung in eine Spalte
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)

        # Berechnung des 'div_term' Tensors
        div_term = torch.pow(10000.0, torch.arange(0, d_model, 2).float() / d_model)
        
        pe[:, 0::2] = #1
        pe[:, 1::2] = #2
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    # Zuschnitt der Positional Encoding Matrix auf die Länge der Eingabe (Token Embeddings)
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# Multi-Head Attention

Wie Ihr gesehen habt wird das Embedding (Sequenzlänge × d_model) zunächst in Q, K und V aufgeteilt, wobei diese drei Matrizen identisch sind. 
Anschliessend werden diese mit einer lernbaren Gewichtsmatrix (linear Layer) (d_model × d_model) multipliziert.

Zudem definieren wir noch eine Gewichtsmatrix derselben Größe für die zusammengefügte attention Matrix. 

### Aufgabe 2
Vervollständige __#3__, __#4__, __#5__ und __#6__. 

__Hinweis: Linear Layers (lernbare Gewichtsmatrizen) werden in pytorch mit nn.Linear() erzeugt, wobei die Dimensionen übergeben werden.__



In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()

        # Es ist notwendig, dass d_model durch num_heads teilbar ist.
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialisierung der Parameter
        self.d_model = d_model # Modell dimension
        self.num_heads = num_heads # Anzahl der Heads
        self.d_k = d_model // num_heads # Dimension von jedem Head von K und Q und V
        
        # Lineare Transformationen für Q, K, V und Output
        self.W_q = #3 # Query Gewichte 
        self.W_k = #4 # Key Gewichte
        self.W_v = #5 # Value Gewichte
        self.W_o = #6 # Output Gewichte
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Attention scores werden berechnet -> Dimension: SL*SL 
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Optional: Maske wird angewendet
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax wird angewendet
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Attention wird berechnet in dem man eine Matrixmultiplikation mit V durchführt  -> Dimension: SL*d_k
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Aufteilen in Heads -> Dimension für jeden Head: SL*d_k
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Heads werden zusammengeführt -> Dimension: SL*d_model
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Lineare Transformationen für Q, K, V und aufteilen in Heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Attention wird berechnet
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Heads werden zusammengeführt und lineare Transformation für Output
        output = self.W_o(self.combine_heads(attn_output))
        return output

# Feed Forward Layer

Die Feed Forward Layer folgt folgendem Schema: 

$$
FFN(x) = \max(0, xW_1 + b_1)W_2 + b_2
$$

Es wird also der Relu-Output der ersten linearen Transformation mit der zweiten linearen Transformation multipliziert. 
### Aufgabe 3

Vervollständige __#7__. 

__Hinweis: Um den Input für eine lineare Transformation in pytorch zu definieren kannst du auch den Output einer anderen linearen Transformation übergeben. Die ReLU kannst du anwenden indem du der Instanz eine Matrix übergibst__
_______________________________________________________________________________________________________
__Beispiel:__ 

self.Linear1(self.Linear2(x))

Hier wird  Linear1 der Output von Linear2 übergeben. Vorrausgesetzt die Dimensionen stimmen.
________________________________________________________________________________________________________


In [4]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        # Lineare Transformation -> Dimension: d_model x d_ff
        self.fc1 = nn.Linear(d_model, d_ff)
        # Lineare Transformation -> Dimension: d_ff x d_model
        self.fc2 = nn.Linear(d_ff, d_model)
        # Relu Aktivierungsfunktion
        self.relu = nn.ReLU()

    def forward(self, x):
        return #7
        

# Zusammenbau der Encoder Layer

Wie das Schaubild zeigt besteht der Encoder aus der MultiHeadAttention Layer sowie einer FeedForward Layer und 2 Normalisierungen. 

Der folgende Code implementiert den Encoder. 

__Hinweis:__
Der Output der MultiHeadAttention Layer sowie die Positional Encodings werden in der Normalization Layer übergeben. 
Das passiert hier mit einem '+'' obwohl man vielleicht ein ',' erwarten würde. 
Das liegt an einer sogenannten Residualverbindung.
Ohne darauf genauer einzugehen handelt sich dabei um eine Technik um den Vanishing oder Exploding Gradient zu vermeiden. 

In [5]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        # Die Komponenten des Encoders werden initialisiert
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        # Eine Dropout-Schicht wird hinzugefügt um u.A Overfitting zu vermeiden
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        # Die Attention wird berechnet
        attn_output = self.self_attn(x, x, x, mask)
        # Erste Normalisierung
        x = self.norm1(x + self.dropout(attn_output))
        # Feed Forward Netzwerk wird durchlaufen
        ff_output = self.feed_forward(x)
        # Zweite Normalisierung
        x = self.norm2(x + self.dropout(ff_output))
        return x

# Zusammenbau der Decoder Layer

Analog zum Decoder. Die Schichten werden nacheinander implementiert. 

Zu unterscheiden ist hier zwischen der self-Attention und der cross-Attention. Während die self-attention im Decoder ähnlich zur self-Attention im Encoder funktioniert (bloß mit Maske), kombiniert die cross-Attention den Output des Encoders mit dem Output der self-Attention Layer. 

In [6]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        # Die Komponenten des Decoders werden initialisiert
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        # Das Herzsück des Decoders, die Cross-Attention
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        # Eine Dropout-Schicht wird hinzugefügt um u.A Overfitting zu vermeiden
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        # Die Attention wird berechnet mit einer Maske
        attn_output = self.self_attn(x, x, x, tgt_mask)
        # Erste Normalisierung
        x = self.norm1(x + self.dropout(attn_output))
        # Die Cross-Attention wird berechnet
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        # Zweite Normalisierung
        x = self.norm2(x + self.dropout(attn_output))
        # Feed Forward Netzwerk wird durchlaufen
        ff_output = self.feed_forward(x)
        # dritte Normalisierung
        x = self.norm3(x + self.dropout(ff_output))
        return x

# Zusammenbau des Transformers

__Das Modell wird hier zusammengebaut__ 

Hier muss nichts implementiert werden!

In [7]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()

        # Definierung der Embeddings für Encoder und Decoder für jeden distinkten Token
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        
        # Positional Encoding wird definiert
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        # Initialisierung der n-Encoder- und n-Decoder-Layers
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        # Lineare Transformation um auf die Zielvokabulargröße zu projektieren
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    # Maske wird generiert
    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask


    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        
        # Embeddings werden erzeugt
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        # Encoder wird mit (Source) Input Embeddings und Maske durchlaufen
        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        # Decoder wird mit (Target) Input Embeddings, Encoder Output und Maske durchlaufen
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        # Lineare Transformation wird durchgeführt um Dimension: SL x d_model -> SL x tgt_vocab_size zu projektieren
        output = self.fc(dec_output)
        return output

# Training mit Dummy-Daten

Um den Trainingsprozess zu veranschaulichen, trainieren wir hier das Transformer Modell mit zwei Sequenzen an zufälligen Zahlen. 

In [8]:
# Parameter laut dem Paper "Attention is All You Need"

src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

#sezen des seeds
torch.manual_seed(42)

# Generieren von Testdaten
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length)) 
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()
# Trainingsschleife
for epoch in range(3):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.695940017700195
Epoch: 2, Loss: 8.565486907958984
Epoch: 3, Loss: 8.493195533752441


# Ausblick: Übersetzen mit Transformer Modellen

Gerne dürft ihr den untenstehenden Code ausführen um eine Übersetzung durchzuführen. Der Prozess ist im Grunde derselbe wie oben.

Die Übersetzung wird aufgrund der wenigen Daten natürlich (sehr) schlecht sein, 
man kann aber auch problemlos große Datensätze zum trainieren heranziehen. Falls man Lust hat, kann man das ja in seiner Freizeit mal probieren. 

Mit steigender Anzahl an Epochen steigt natürlich auch die Leistungsfähigkeit des Modells. 


In [9]:
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

train_data = [
    ("Ich kann nicht glauben, dass das wahr ist.", "I can't believe this is true."),
    ("Das Wetter ist heute schön.", "The weather is nice today."),
    ("Ich liebe Programmierung.", "I love programming."),
    ("Wie geht es dir?", "How are you?"),
    ("Das ist ein Test.", "This is a test."),
    ("Die Mensa macht immer gutes Essen.", "The canteen always makes good food."),
    ("Ich bin hungrig.", "I am hungry."),
    ("Was machst du?", "What are you doing?"),
    ("Es ist ein schöner Tag.", "It is a beautiful day."),
    ("Das Essen schmeckt schlecht.", "The food tastes bad."),
    ("Das Essen schmeckt gut.", "The food tastes good."),
    ("Ich gehe zur Schule.", "I am going to school."),
    ("Was hast du heute gelernt?", "What did you learn today?"),
    ("Ich habe eine Katze.", "I have a cat."),
    ("Ich habe einen Hund.", "I have a dog.")
]

src_tokenizer = get_tokenizer('basic_english')
tgt_tokenizer = get_tokenizer('basic_english')


def yield_tokens(data, tokenizer):
    for src_sentence, tgt_sentence in data:
        yield tokenizer(src_sentence)
        yield tokenizer(tgt_sentence)

src_vocab = build_vocab_from_iterator(yield_tokens(train_data, src_tokenizer), specials=['<pad>', '<sos>', '<eos>'])
tgt_vocab = build_vocab_from_iterator(yield_tokens(train_data, tgt_tokenizer), specials=['<pad>', '<sos>', '<eos>'])

src_vocab.set_default_index(src_vocab['<pad>'])
tgt_vocab.set_default_index(tgt_vocab['<pad>'])

def sentence_to_tensor(sentence, vocab, tokenizer):
    tokens = tokenizer(sentence)
    indices = [vocab['<sos>']] + [vocab[token] for token in tokens] + [vocab['<eos>']]
    return torch.tensor(indices).unsqueeze(0)

train_src_tensors = [sentence_to_tensor(src, src_vocab, src_tokenizer) for src, tgt in train_data]
train_tgt_tensors = [sentence_to_tensor(tgt, tgt_vocab, tgt_tokenizer) for src, tgt in train_data]

src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)


transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

criterion = nn.CrossEntropyLoss(ignore_index=src_vocab['<pad>'])
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(20): 
    total_loss = 0
    for src_tensor, tgt_tensor in zip(train_src_tensors, train_tgt_tensors):
        optimizer.zero_grad()
        output = transformer(src_tensor, tgt_tensor[:, :-1])
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_tensor[:, 1:].contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch: {epoch+1}, Loss: {total_loss/len(train_data)}")

    

def translate(transformer, src_sentence, src_vocab, tgt_vocab, max_seq_length):
    transformer.eval()
    
    src_tokens = src_tokenizer(src_sentence)
    src_indices = [src_vocab['<sos>']] + [src_vocab[token] for token in src_tokens] + [src_vocab['<eos>']]
    src_tensor = torch.tensor(src_indices).unsqueeze(0)
    
    tgt_indices = [tgt_vocab['<sos>']]
    tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0)
    
    with torch.no_grad():
        for _ in range(max_seq_length):
            output = transformer(src_tensor, tgt_tensor)
            next_token = output.argmax(2)[:, -1].item()
            tgt_indices.append(next_token)
            if next_token == tgt_vocab['<eos>']:
                break
            tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0)
    
    tgt_tokens = [tgt_vocab.get_itos()[idx] for idx in tgt_indices]
    return ' '.join(tgt_tokens)

/Users/romanoelfken/miniconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/romanoelfken/miniconda3/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/romanoelfken/miniconda3/lib/python3.11/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT

Epoch: 1, Loss: 3.944688892364502
Epoch: 2, Loss: 2.8693776448567707
Epoch: 3, Loss: 1.735778268178304
Epoch: 4, Loss: 1.1560726404190063
Epoch: 5, Loss: 0.7502215166886648
Epoch: 6, Loss: 0.6137418289979298
Epoch: 7, Loss: 0.49692387382189435
Epoch: 8, Loss: 0.46944422721862794
Epoch: 9, Loss: 0.47423750956853233
Epoch: 10, Loss: 0.3201397856076558
Epoch: 11, Loss: 0.24799330482880275
Epoch: 12, Loss: 0.16066059619188308
Epoch: 13, Loss: 0.11286518834531307
Epoch: 14, Loss: 0.0669271974513928
Epoch: 15, Loss: 0.05244739806900422
Epoch: 16, Loss: 0.05275492804745833
Epoch: 17, Loss: 0.054446007745961346
Epoch: 18, Loss: 0.024927052296698095
Epoch: 19, Loss: 0.014441488745311896
Epoch: 20, Loss: 0.011865517341842253


In [12]:
test_sentence = "Ich habe einen Hund"
translated_sentence = translate(transformer, test_sentence, src_vocab, tgt_vocab, max_seq_length)
print(f"Translated Sentence: {translated_sentence}")

Translated Sentence: <sos> i have a dog . <eos>
